In [1]:
import os
os.getcwd()
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)

In [2]:
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from vae_earlystopping import EarlyStopping
from model.m2_bce import BCEcVAE
from model.m2_mse import MSEcVAE
from loss.l2_bce import l2_bce
from loss.l2_mse import l2_mse
import joblib
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import json

In [3]:
# 결과 저장용
results = {
    "R2_With_React": [],
    "R2_Without_React": []
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_and_evaluate(c_path, x_data, seed):
    """
    특정 조건 데이터(c_path)를 받아 BCE+MSE 결합 모델의 R2 Score를 반환하는 함수
    """
    c_data = np.load(c_path)
    
    # 데이터 분할
    x_train, x_temp, c_train, c_temp = train_test_split(x_data, c_data, random_state=seed, test_size=0.4)
    x_val, x_test, c_val, c_test = train_test_split(x_temp, c_temp, random_state=seed, test_size=0.5)
    
    # 스케일링
    x_scaler, c_scaler = MinMaxScaler(), MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val, x_test = x_scaler.transform(x_val), x_scaler.transform(x_test)
    c_val, c_test = c_scaler.transform(c_val), c_scaler.transform(c_test)
    
    # Loader 생성
    t_loader = DataLoader(TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(c_train)), batch_size=64, shuffle=True)
    v_loader = DataLoader(TensorDataset(torch.FloatTensor(x_val), torch.FloatTensor(c_val)), batch_size=64)
    test_loader = DataLoader(TensorDataset(torch.FloatTensor(x_test), torch.FloatTensor(c_test)), batch_size=64)
    
    x_dim, c_dim = x_train.shape[1], c_train.shape[1]

    # --- 1. BCE 모델 학습 ---
    model_bce = BCEcVAE(x_dim, c_dim, z_dim=8).to(device)
    opt_bce = optim.Adam(model_bce.parameters(), lr=1e-3, weight_decay=1e-5)
    es_bce = EarlyStopping(patience=40, min_delta=1e-9)
    
    for epoch in range(1, 801):
        model_bce.train()
        for x, c in t_loader:
            x, c = x.to(device), c.to(device)
            opt_bce.zero_grad()
            logit, mu, logvar = model_bce(x, c)
            loss = l2_bce(logit, x, mu, logvar)['loss']
            loss.backward(); opt_bce.step()
        
        # Validation (생략 가능하나 조기종료 위해 유지)
        model_bce.eval()
        val_loss = 0
        with torch.no_grad():
            for vx, vc in v_loader:
                vx, vc = vx.to(device), vc.to(device)
                vl, vm, vv = model_bce(vx, vc)
                val_loss += l2_bce(vl, vx, vm, vv)['loss'].item()
        if es_bce(val_loss/len(v_loader), model_bce): break
    
    es_bce.load_best_model(model_bce)

    # --- 2. MSE 모델 학습 ---
    model_mse = MSEcVAE(x_dim, c_dim, z_dim=8).to(device)
    opt_mse = optim.Adam(model_mse.parameters(), lr=1e-3, weight_decay=1e-5)
    es_mse = EarlyStopping(patience=40, min_delta=1e-9)
    
    for epoch in range(1, 801):
        model_mse.train()
        for x, c in t_loader:
            x, c = x.to(device), c.to(device)
            opt_mse.zero_grad()
            xh, mu, logvar = model_mse(x, c)
            loss = l2_mse(xh, x, mu, logvar)['loss']
            loss.backward(); opt_mse.step()
            
        model_mse.eval()
        val_loss = 0
        with torch.no_grad():
            for vx, vc in v_loader:
                vx, vc = vx.to(device), vc.to(device)
                xh, vm, vv = model_mse(vx, vc)
                val_loss += l2_mse(xh, vx, vm, vv)['loss'].item()
        if es_mse(val_loss/len(v_loader), model_mse): break
        
    es_mse.load_best_model(model_mse)

    # --- 3. 최종 평가 (BCE_MSE 가중합) ---
    model_bce.eval(); model_mse.eval()
    bce_logits, mse_preds, y_true = [], [], []
    
    with torch.no_grad():
        for xt, ct in test_loader:
            xt, ct = xt.to(device), ct.to(device)
            b_logit, _, _ = model_bce(xt, ct)
            m_pred, _, _ = model_mse(xt, ct)
            bce_logits.append(b_logit.cpu().numpy())
            mse_preds.append(m_pred.cpu().numpy())
            y_true.append(xt.cpu().numpy())
            
    bce_prob = 1 / (1 + np.exp(-np.vstack(bce_logits))) # Sigmoid
    x_hat_raw = x_scaler.inverse_transform(np.vstack(mse_preds))
    x_true_phys = x_scaler.inverse_transform(np.vstack(y_true))
    
    # Soft weighting 적용
    final_pred = x_hat_raw * bce_prob
    return r2_score(x_true_phys.flatten(), final_pred.flatten())

# --- 메인 실험 루프 ---
x_data = np.load('./data/metal.npy')
seeds = np.random.randint(1, 1000, size=20)

for i, seed in enumerate(seeds):
    print(f"\n[Iteration {i+1}/20] Seed: {seed}")
    
    # Case 1: With Reaction
    r2_with = train_and_evaluate('./data/pre_re_change_temp_logconst.npy', x_data, seed)
    results["R2_With_React"].append(float(r2_with))
    
    # Case 2: Without Reaction
    r2_without = train_and_evaluate('./data/pre_re_change_temp_no_react_const.npy', x_data, seed)
    results["R2_Without_React"].append(float(r2_without))
    
    print(f"Result -> With: {r2_with:.4f} | Without: {r2_without:.4f}")

# 결과 저장
with open("./case1_logreaction_constant_results.json", "w") as f:
    json.dump(results, f, indent=2)


[Iteration 1/20] Seed: 182
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 ou

C:\Users\User\AppData\Local\Temp\ipykernel_32876\3363428110.py:97: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-np.vstack(bce_logits))) # Sigmoid


EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counte

C:\Users\User\AppData\Local\Temp\ipykernel_32876\3363428110.py:97: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-np.vstack(bce_logits))) # Sigmoid


EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counte

...

C:\Users\User\AppData\Local\Temp\ipykernel_32876\3363428110.py:97: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-np.vstack(bce_logits))) # Sigmoid


EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counte

C:\Users\User\AppData\Local\Temp\ipykernel_32876\3363428110.py:97: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-np.vstack(bce_logits))) # Sigmoid


EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 6 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counte

C:\Users\User\AppData\Local\Temp\ipykernel_32876\3363428110.py:97: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-np.vstack(bce_logits))) # Sigmoid


In [4]:
save_path = "./results_logreaction_constant.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Saved:", save_path)

Saved: ./results_logreaction_constant.json
